# Project: Wrangling WeRateDogs' Enhanced Twitter Archive

### Scope
---------------------------------------------------------
WeRateDogs (later referred to as WRD in this document) is a Twitter account that rates people's dogs with a humorous comment about the dog. These ratings almost always have a denominator of 10. The numerators, though? Almost always greater than 10. 11/10, 12/10, 13/10, etc. Why? Because "they're good dogs Brent." WRD has over 4 million followers and has received international media coverage.

This project focuses on actualizing and accentuating the three data wrangling techniques on the WRD Twitter archive. WRD downloaded their Twitter archive and sent it to Udacity via email exclusively for use in this project. The archive contains basic tweet data (tweet ID, timestamp, text, etc.) for all 5000+ of their tweets as they stood on August 1, 2017.

### Modules used in this notebook:
* Pandas
* Numpy
* Matplotlib
* Seaborn
* Tweepy
* Requests
* Plotly
* Datetime
* Pydash

In [1104]:
pip install python-dotenv tweepy pydash

Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: python-dotenv in c:\users\tevinaduma\anaconda3\lib\site-packages (0.20.0)



----

#### Imports

In [1105]:
import requests
import tweepy as twpy
from dotenv import load_dotenv
import timeit
import datetime as dt
import _json

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

from pydash import at
import plotly.express as px

%matplotlib inline

----

### Gather

#### This project involves obtaining three seperate datasets from various sources. I will be using different methods to obtain each dataset as specified below. 

This custom function built from Pandas' `read_csv()` method will be used to read various datasets. The `header` and `names` parameters will override each other depending on which of the two is set to `None`

In [1106]:
def open_set(csv, sep=',', header=0, names=[]):
    df = pd.read_csv(csv, low_memory=False, sep=sep, names=names, header=header)
    
    return df

**`twitter-archive-enhanced` table**

WRD's Twitter archive data was provided by Udacity and **downloaded manually through the Chrome browser.** I will import the data locally from my storage.

In [1107]:
df_tw_arch = open_set('data/twitter-archive-enhanced.csv', header=0, names=None)
df_tw_arch.head()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None


----

**`image-predictions` table**

The `image_predictions.tsv` file is present in each tweet according to a neural network. It is hosted on Udacity's servers and will be **downloaded programmatically using the Requests library**. 

The content obtained from the file hosted on the url will be written into a file on the local machine using Python's **open()** method. 

In [1108]:
url = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'

* Use **`requests.get()`** to obtain the data from the url.
* Parse the content into a new file named `image-predictions.tsv`. (Note the file is opened using `wb` since the content obtained is returned in byte format)

In [1109]:
r = requests.get(url)

try:
    f = open('image-predictions.tsv', 'wb')
    f.write(r.content)
    
finally:
    f.close()

ConnectionError: HTTPSConnectionPool(host='d17h27t6h515a5.cloudfront.net', port=443): Max retries exceeded with url: /topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x0000014953D61A60>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))

* tsv stands for tab-separated-values so it would make sense to specify tabs (`\t`) as the separator in Pandas' `read_csv() method`

In [ ]:
df_image_pred = open_set('data/image-predictions.tsv', sep='\t', names=None, header=0)
df_image_pred.head()

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True


----

#### **Obtaining additional tweet data from `Twitter API` using Python's `Tweepy`** module

For our final dataset, we will be using the Twitter API and Python's Tweepy library to query Twitter for each tweet's **retweet_count**, **favorite_count**, **geo data** and **language data**. These attributes will later be used to generate insights.

**Note that you need either a combination of your consumer_key, consumer_secret, access_key and access_key secret or a bearer_token to query data through the Twitter API.**

In [ ]:
# Obtain credentials
load_dotenv('.env')

True

I put to use Python's `dotenv` library to cache my `bearer_token` from a secret file and obtain it's value using the `os.getenv` method.

I opted for a custom function to obtain tweet data through Tweepy's [**_get_status()_**](http://docs.tweepy.org/en/v3.5.0/api.html) method. The `tweet_id` from  the `df_tw_arch` dataset will be converted to a list and passed into the function. Each tweet's JSON data that I require for this project will be parsed into a new file (`tweet_json.txt`) and appended one after the other.

There are a few tweets and retweets that may have been deleted since WRD's submission of their archive. I have used a `try-except` block to capture their **tweet_id** into a separate array for later analysis.

In [ ]:
# custom to extract tweet data
def get_tweets(ids):
    
    # Authorization to bearer_token
    auth = twpy.OAuth2BearerHandler(os.getenv('BEARER_TOKEN'))
    
    # Calling api
    api = twpy.API(auth, wait_on_rate_limit = True)
    
    # Empty Array
    del_tweets = []
    
    # Start a code timer for the loop
    start = timeit.timeit()  
    for tw_id in ids: 
        try:
            tw_status = api.get_status(tw_id, tweet_mode='extended')._json
            try:
                f = open('tweet_json.txt', 'a+', encoding='utf-8')
                f.write(f"{tw_status['id']},{tw_status['retweet_count']},{tw_status['favorite_count']},{tw_status['geo']},{tw_status['lang']}\n")
            finally:
                f.close()
            rt_count = tw_status['retweet_count']
            fv_count = tw_status['favorite_count']
                        
            print(f'This tweet -> {tw_id} has {rt_count} retweets and {fv_count} likes')
        except Exception as e:
            print(f'This tweet -> {tw_id} has been deleted')
            del_tweets.append({'tweet_id': tw_id})
    
    # Stop the code timer for our loop
    end = timeit.timeit()
    
    # Calculate how long it took to run
    print(f'This code took {end - start} to run')
    
    # Print out the deleted tweet_ids
    return (f'These are the deleted tweet_ids:\n{del_tweets}')

* Capture all the data in the `tweet_id` column of `df_tw_arch` into a list

In [ ]:
tw_ids = list(df_tw_arch.tweet_id)
tw_ids

[892420643555336193,
 892177421306343426,
 891815181378084864,
 891689557279858688,
 891327558926688256,
 891087950875897856,
 890971913173991426,
 890729181411237888,
 890609185150312448,
 890240255349198849,
 890006608113172480,
 889880896479866881,
 889665388333682689,
 889638837579907072,
 889531135344209921,
 889278841981685760,
 888917238123831296,
 888804989199671297,
 888554962724278272,
 888202515573088257,
 888078434458587136,
 887705289381826560,
 887517139158093824,
 887473957103951883,
 887343217045368832,
 887101392804085760,
 886983233522544640,
 886736880519319552,
 886680336477933568,
 886366144734445568,
 886267009285017600,
 886258384151887873,
 886054160059072513,
 885984800019947520,
 885528943205470208,
 885518971528720385,
 885311592912609280,
 885167619883638784,
 884925521741709313,
 884876753390489601,
 884562892145688576,
 884441805382717440,
 884247878851493888,
 884162670584377345,
 883838122936631299,
 883482846933004288,
 883360690899218434,
 883117836046

* Pass the list of tweet_ids into our custom function `get_tweets()`

In [ ]:
get_tweets(tw_ids)

This tweet -> 892420643555336193 has 7010 retweets and 33829 likes
This tweet -> 892177421306343426 has 5301 retweets and 29340 likes
This tweet -> 891815181378084864 has 3482 retweets and 22070 likes
This tweet -> 891689557279858688 has 7228 retweets and 36953 likes
This tweet -> 891327558926688256 has 7765 retweets and 35324 likes
This tweet -> 891087950875897856 has 2602 retweets and 17815 likes
This tweet -> 890971913173991426 has 1666 retweets and 10370 likes
This tweet -> 890729181411237888 has 15761 retweets and 56891 likes
This tweet -> 890609185150312448 has 3624 retweets and 24530 likes
This tweet -> 890240255349198849 has 6101 retweets and 27973 likes
This tweet -> 890006608113172480 has 6154 retweets and 27051 likes
This tweet -> 889880896479866881 has 4168 retweets and 24574 likes
This tweet -> 889665388333682689 has 8354 retweets and 42073 likes
This tweet -> 889638837579907072 has 3718 retweets and 23688 likes
This tweet -> 889531135344209921 has 1884 retweets and 13358 

This tweet -> 868622495443632128 has 4474 retweets and 23656 likes
This tweet -> 868552278524837888 has 1754 retweets and 8972 likes
This tweet -> 867900495410671616 has 3574 retweets and 21620 likes
This tweet -> 867774946302451713 has 6258 retweets and 30288 likes
This tweet -> 867421006826221569 has 2118 retweets and 14355 likes
This tweet -> 867072653475098625 has 101 retweets and 0 likes
This tweet -> 867051520902168576 has 6792 retweets and 28728 likes
This tweet -> 866816280283807744 has been deleted
This tweet -> 866720684873056260 has 4133 retweets and 17882 likes
This tweet -> 866686824827068416 has 2973 retweets and 17143 likes
This tweet -> 866450705531457537 has 30222 retweets and 108951 likes
This tweet -> 866334964761202691 has 12218 retweets and 46650 likes
This tweet -> 866094527597207552 has 7172 retweets and 0 likes
This tweet -> 865718153858494464 has 4853 retweets and 22948 likes
This tweet -> 865359393868664832 has 4295 retweets and 23627 likes
This tweet -> 86500

This tweet -> 844979544864018432 has 2318 retweets and 12701 likes
This tweet -> 844973813909606400 has 2862 retweets and 13955 likes
This tweet -> 844704788403113984 has been deleted
This tweet -> 844580511645339650 has 2822 retweets and 15313 likes
This tweet -> 844223788422217728 has 1994 retweets and 12721 likes
This tweet -> 843981021012017153 has 2703 retweets and 14155 likes
This tweet -> 843856843873095681 has 4176 retweets and 19998 likes
This tweet -> 843604394117681152 has 2485 retweets and 15740 likes
This tweet -> 843235543001513987 has 5430 retweets and 20013 likes
This tweet -> 842892208864923648 has been deleted
This tweet -> 842846295480000512 has 3319 retweets and 14245 likes
This tweet -> 842765311967449089 has 1177 retweets and 6304 likes
This tweet -> 842535590457499648 has 3191 retweets and 16953 likes
This tweet -> 842163532590374912 has 5274 retweets and 22782 likes
This tweet -> 842115215311396866 has 2764 retweets and 13061 likes
This tweet -> 8418339930205388

This tweet -> 828011680017821696 has 2000 retweets and 9786 likes
This tweet -> 827933404142436356 has 4843 retweets and 18879 likes
This tweet -> 827653905312006145 has 2798 retweets and 14611 likes
This tweet -> 827600520311402496 has 865 retweets and 7064 likes
This tweet -> 827324948884643840 has 2812 retweets and 14970 likes
This tweet -> 827228250799742977 has been deleted
This tweet -> 827199976799354881 has 2064 retweets and 10012 likes
This tweet -> 826958653328592898 has 4616 retweets and 20325 likes
This tweet -> 826848821049180160 has 9356 retweets and 34080 likes
This tweet -> 826615380357632002 has 3686 retweets and 0 likes
This tweet -> 826598799820865537 has 230 retweets and 4865 likes
This tweet -> 826598365270007810 has 2183 retweets and 9460 likes
This tweet -> 826476773533745153 has 3896 retweets and 17394 likes
This tweet -> 826240494070030336 has 2423 retweets and 12540 likes
This tweet -> 826204788643753985 has 856 retweets and 4602 likes
This tweet -> 8261152722

This tweet -> 813081950185472002 has 2598 retweets and 9422 likes
This tweet -> 813066809284972545 has 1830 retweets and 7545 likes
This tweet -> 813051746834595840 has 6867 retweets and 19931 likes
This tweet -> 812781120811126785 has 1787 retweets and 7158 likes
This tweet -> 812747805718642688 has been deleted
This tweet -> 812709060537683968 has 1353 retweets and 6297 likes
This tweet -> 812503143955202048 has 1148 retweets and 5745 likes
This tweet -> 812466873996607488 has 1802 retweets and 7569 likes
This tweet -> 812372279581671427 has 3427 retweets and 12946 likes
This tweet -> 811985624773361665 has 1312 retweets and 6920 likes
This tweet -> 811744202451197953 has 1479 retweets and 7165 likes
This tweet -> 811647686436880384 has 682 retweets and 5323 likes
This tweet -> 811627233043480576 has 2856 retweets and 12084 likes
This tweet -> 811386762094317568 has 5989 retweets and 19887 likes
This tweet -> 810984652412424192 has 1335 retweets and 5062 likes
This tweet -> 810896069

...

This tweet -> 674790488185167872 has 219 retweets and 986 likes
This tweet -> 674788554665512960 has 181 retweets and 719 likes
This tweet -> 674781762103414784 has 1050 retweets and 1808 likes
This tweet -> 674774481756377088 has 412 retweets and 991 likes
This tweet -> 674767892831932416 has 692 retweets and 1634 likes
This tweet -> 674764817387900928 has 200 retweets and 700 likes
This tweet -> 674754018082705410 has 384 retweets and 1229 likes
This tweet -> 674752233200820224 has 402 retweets and 1314 likes
This tweet -> 674743008475090944 has 469 retweets and 1261 likes
This tweet -> 674742531037511680 has 45 retweets and 430 likes
This tweet -> 674739953134403584 has 327 retweets and 967 likes
This tweet -> 674737130913071104 has 80 retweets and 580 likes
This tweet -> 674690135443775488 has 413 retweets and 1027 likes
This tweet -> 674670581682434048 has 569 retweets and 1450 likes
This tweet -> 674664755118911488 has 205 retweets and 812 likes
This tweet -> 674646392044941312 h

This tweet -> 672231046314901505 has 876 retweets and 1624 likes
This tweet -> 672222792075620352 has 176 retweets and 703 likes
This tweet -> 672205392827572224 has 1010 retweets and 1994 likes
This tweet -> 672169685991993344 has 320 retweets and 908 likes
This tweet -> 672160042234327040 has 304 retweets and 766 likes
This tweet -> 672139350159835138 has 597 retweets and 1548 likes
This tweet -> 672125275208069120 has 993 retweets and 2120 likes
This tweet -> 672095186491711488 has 328 retweets and 879 likes
This tweet -> 672082170312290304 has 319 retweets and 840 likes
This tweet -> 672068090318987265 has 440 retweets and 1141 likes
This tweet -> 671896809300709376 has 3679 retweets and 7631 likes
This tweet -> 671891728106971137 has 484 retweets and 1176 likes
This tweet -> 671882082306625538 has 1215 retweets and 3117 likes
This tweet -> 671879137494245376 has 575 retweets and 1249 likes
This tweet -> 671874878652489728 has 479 retweets and 1107 likes
This tweet -> 6718663421826

This tweet -> 669993076832759809 has 70 retweets and 288 likes
This tweet -> 669972011175813120 has 134 retweets and 396 likes
This tweet -> 669970042633789440 has 43 retweets and 272 likes
This tweet -> 669942763794931712 has 133 retweets and 445 likes
This tweet -> 669926384437997569 has 81 retweets and 339 likes
This tweet -> 669923323644657664 has 50 retweets and 202 likes
This tweet -> 669753178989142016 has 341 retweets and 710 likes
This tweet -> 669749430875258880 has 55 retweets and 229 likes
This tweet -> 669684865554620416 has 75 retweets and 448 likes
This tweet -> 669683899023405056 has 94 retweets and 339 likes
This tweet -> 669682095984410625 has 116 retweets and 312 likes
This tweet -> 669680153564442624 has 245 retweets and 588 likes
This tweet -> 669661792646373376 has 367 retweets and 715 likes
This tweet -> 669625907762618368 has 1573 retweets and 3130 likes
This tweet -> 669603084620980224 has 316 retweets and 846 likes
This tweet -> 669597912108789760 has 133 retw

This tweet -> 667502640335572993 has 194 retweets and 485 likes
This tweet -> 667495797102141441 has 232 retweets and 458 likes
This tweet -> 667491009379606528 has 190 retweets and 459 likes
This tweet -> 667470559035432960 has 85 retweets and 225 likes
This tweet -> 667455448082227200 has 53 retweets and 171 likes
This tweet -> 667453023279554560 has 74 retweets and 279 likes
This tweet -> 667443425659232256 has 490 retweets and 682 likes
This tweet -> 667437278097252352 has 199 retweets and 396 likes
This tweet -> 667435689202614272 has 76 retweets and 266 likes
This tweet -> 667405339315146752 has 197 retweets and 412 likes
This tweet -> 667393430834667520 has 50 retweets and 168 likes
This tweet -> 667369227918143488 has 145 retweets and 328 likes
This tweet -> 667211855547486208 has 210 retweets and 425 likes
This tweet -> 667200525029539841 has 228 retweets and 546 likes
This tweet -> 667192066997374976 has 86 retweets and 332 likes
This tweet -> 667188689915760640 has 333 retwe

[{'tweet_id': 888202515573088257},
 {'tweet_id': 873697596434513921},
 {'tweet_id': 872668790621863937},
 {'tweet_id': 872261713294495745},
 {'tweet_id': 869988702071779329},
 {'tweet_id': 866816280283807744},
 {'tweet_id': 861769973181624320},
 {'tweet_id': 856602993587888130},
 {'tweet_id': 856330835276025856},
 {'tweet_id': 851953902622658560},
 {'tweet_id': 851861385021730816},
 {'tweet_id': 845459076796616705},
 {'tweet_id': 844704788403113984},
 {'tweet_id': 842892208864923648},
 {'tweet_id': 837366284874571778},
 {'tweet_id': 837012587749474308},
 {'tweet_id': 829374341691346946},
 {'tweet_id': 827228250799742977},
 {'tweet_id': 812747805718642688},
 {'tweet_id': 802247111496568832},
 {'tweet_id': 779123168116150273},
 {'tweet_id': 775096608509886464},
 {'tweet_id': 771004394259247104},
 {'tweet_id': 770743923962707968},
 {'tweet_id': 766864461642756096},
 {'tweet_id': 759923798737051648},
 {'tweet_id': 759566828574212096},
 {'tweet_id': 754011816964026368},
 {'tweet_id': 699370

> * We will then read the data from `tweet_json.txt` using either one of Pandas' `read_csv()` or `read_table()` methods. I will specify the column tags using the `names` parameter.

In [ ]:
df_tw_data = open_set('data/tweet_json.txt', names=['tweet_id', 'retweet_count', 'favorite_count', 'geo_data', 'lang_data'], header=None)

> * To later assess the data obtained from the Twitter api visually using Excel/Sheets, I will export it to a csv file.

In [ ]:
df_tw_data.to_csv('data/tw_data.csv',index=False)
df_tw_data

,tweet_id,retweet_count,favorite_count,geo_data,lang_data
0,892420643555336193,7010,33829,None,en
1,892177421306343426,5301,29340,None,en
2,891815181378084864,3482,22070,None,en
3,891689557279858688,7228,36953,None,en
4,891327558926688256,7765,35324,None,en
...,...,...,...,...,...
2320,666049248165822465,37,89,None,en
2321,666044226329800704,115,247,None,en
2322,666033412701032449,36,100,None,en
2323,666029285002620928,39,112,None,en


----

### Assess

I will use a spreadsheet program for my visual assessment (Google Sheets, MS Excel etc) and employ Pandas' library for my programmatic assessment.

#### `df_tw_arch` dataset

##### Visual Assessment
----

![alt](img/tw_arch.png)

> * There are a few columns with glaringly empty fields: `in_reply_to_status`, `in_reply_to_user_id`, `retweeted_status_user_id`, `retweeted_status_timestamp` **(Quality Issue)**
> * At a glance, `rating_denominator` seems to have only one unique value and hence no insights can be obtained from that column. I will investigate this further. **(Quality issue)**
>* The four columns describing the "stage" the dog is in should be transposed into one column. **(Tidiness issue)**

##### Programmatic Assessment
<hr>

In [ ]:
df_tw_arch.head()

 > * The `expanded_url` column is not required for this study as it just the full hyperlink to WRD's tweets. **(Tidiness issue)**
 > * The `source` column should be a string of the source from which the data was obtained and not a hyperlink i.e. Twitter, Vine, Tweetdeck **(Tidiness)**

> Describe these next lines HERE PLEASE!!!

In [ ]:
for col in df_tw_arch.columns:
    print(f'The "{col}" column has {df_tw_arch[col].nunique()} unique values')

In [ ]:
df_tw_arch.source.unique()

In [ ]:
df_tw_arch.describe()

> * Based on WRD's rating convention, the mean of `rating_denominator`should not exceed 10. **(Quality Issue)**

In [ ]:
df_tw_arch.rating_numerator.unique()

array([  13,   12,   14,    5,   17,   11,   10,  420,  666,    6,   15,
        182,  960,    0,   75,    7,   84,    9,   24,    8,    1,   27,
          3,    4,  165, 1776,  204,   50,   99,   80,   45,   60,   44,
        143,  121,   20,   26,    2,  144,   88], dtype=int64)

> * There are few ratings that are well out the scope described by WRD's rating system i.e 420, 666, 182, 960, 165, 1776, 204, 143, 121 etc **(Quality issue)**

In [ ]:
df_tw_arch.rating_denominator.unique()

array([ 10,   0,  15,  70,   7,  11, 150, 170,  20,  50,  90,  80,  40,
       130, 110,  16, 120,   2], dtype=int64)

> * There are few rating_denominators that are well out the scope described by WRD's rating system i.e 150, 130, 110, 120, 70, 0, 50 etc **(Quality issue)**

In [ ]:
i = 0
rm = []
rs = []
while i < 2355:
    result = df_tw_arch.rating_denominator[i] != 10
    if result:
        rm.append(i)
    i += 1
df_tw_arch.iloc[rm]

> * The records above have been affected by entry issues in the `rating_numerator` and `rating_denominator` columns. **(Quality issue)**

In [ ]:
df_tw_arch.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   tweet_id                    2356 non-null   int64  
 1   in_reply_to_status_id       78 non-null     float64
 2   in_reply_to_user_id         78 non-null     float64
 3   timestamp                   2356 non-null   object 
 4   source                      2356 non-null   object 
 5   text                        2356 non-null   object 
 6   retweeted_status_id         181 non-null    float64
 7   retweeted_status_user_id    181 non-null    float64
 8   retweeted_status_timestamp  181 non-null    object 
 9   expanded_urls               2297 non-null   object 
 10  rating_numerator            2356 non-null   int64  
 11  rating_denominator          2356 non-null   int64  
 12  name                        2356 non-null   object 
 13  doggo                       2356 

> * The `timestamp` column's datatype should be altered to datetime **(Quality issue)**

In [ ]:
df_tw_arch.isnull().sum()

> * As expected from the visual assessment made, the `in_reply_to_status_id`, `in_reply_to_user_id`, `retweeted_status_id`, `retweeted_status_id`, `retweeted_status_user_id` and `retweeted_status_timestamp` have a lot of null entries **(Quality issue)**

In [ ]:
df_tw_arch.name.value_counts()

>* A number of dog names were not entered making it difficult to analyze whether a dog's name has any influence over WRD's given rating

In [ ]:
df_tw_arch.tweet_id.value_counts()

892420643555336193    1
687102708889812993    1
687826841265172480    1
687818504314159109    1
687807801670897665    1
                     ..
775085132600442880    1
774757898236878852    1
774639387460112384    1
774314403806253056    1
666020888022790149    1
Name: tweet_id, Length: 2356, dtype: int64

> * There are no duplicated records in this dataset 🙂
----

> #### `df_tw_data` dataset

##### Visual Assessment

![](./img/tw_data.png)

> * The `geo_data` column seem to have just one unique entries. Generating any meaningful insights will seemingly prove futile from this column. **(Quality)**

##### Programmatic Assessment

In [ ]:
df_tw_data.head()

In [ ]:
df_tw_data.tail()

> * The main issue is this dataset SHOULD be concatenated into the main twitter archive `df_tw_arch` dataset to gauge how much interaction each dog's WRD tweet. **(Tidiness issue)**  

In [ ]:
df_tw_data.info()

* There are no issues with data types and no null entries in the dataset.

In [ ]:
df_tw_data.duplicated().sum()

0

> * There are no duplicated reords in this dataset 🙂

In [ ]:
df_tw_data.describe()

> * There seems to be a huge disparity in the `favorite_count` column. The range between the min and the first quartile is absurdly large. **(Quality)**

In [ ]:
df_tw_data.nunique()

In [ ]:
df_tw_data.lang_data.unique()

> * The `lang_data` column describes languages supported by Twitter for websites widgets and buttons contained in this dataset. It is a basic pointer as to which language the user is most likely a speaker of. Twitter for Websites will extract the most appropriate language from its position in the DOM tree, if no language is provided in the widget markup. 
<br>
<br>
> The languages encoded in this dataset include:<br>                                   
  English `en` <br>
  Spanish `es` <br>
  Romanian `ro` <br>
  Dutch `nl` <br>
  Indonesian `in` <br>
  Tagalog `tl` <br>
  Estonian `et`<br>
  Basque `eu`<br>
  `und` i.e. `Undefined` is used for cases where a language code was not provided <br>
<br>
> For more info, read [Supported languages and browsers](https://developer.twitter.com/en/docs/twitter-for-websites/supported-languages) on Twitter's Developer Platform.
<hr>

> #### `df_image_pred` dataset

##### Visual Assessment

![](./img/img-pred.png)

> * Some rows have no `True` prediction values and will be rendered used in any EDA involving dog species as a factor. **(Quality issue)**
> * Predictions should be represented as a percentage rather than a float with multiple decimal numbers. **(Quality issue)**
> * The `p1_conf` should be the only prediction value retained since it is the closest to 1 (it is the most trustworthy "dog identifier") in the dataset. **(Tidiness issue)** 
> * `img_num` column is unnecessary since we have links to the images. **(Tidiness issue)**  

##### Programmatic Assessment

In [ ]:
df_image_pred.head()

In [ ]:
df_image_pred.tail()

In [ ]:
df_image_pred.info()

* The `p1`, `p2`, `p3` along with their respective `conf` columns will should be renamed with more descriptive names.

In [ ]:
df_image_pred.describe()

> * The mean values for the `conf` scores align with our visual assessment regarding which one of the columns to retain based on it's closeness to a `TRUE` prediction i.e. p1_conf.

In [ ]:
df_image_pred.nunique()

In [ ]:
df_image_pred.p1.value_counts()

> * A number of predictions are not dog species. These will have to be filtered out of the dataset to ensure we have only records that are dog species. **(Quality Issue)**
----

### Clean
> * I will make a copy of each dataset and clean them separately going through every issue and tackling them with code, and testing to see if my solutions crafted the datasets into the form tha I require.
----

In [1144]:
df_copy_arch = df_tw_arch.copy()
df_copy_data = df_tw_data.copy()
df_copy_img = df_image_pred.copy()

> `df_tw_arch`

##### Define

> **(Tidiness issue)**: Some of the fields in the table bring about repetitiveness and provide no insights at all. I chose to drop these fields due to the following reasons:
>* `in_reply_to_status_id`, `in_reply_to_user_id`,`retweeted_status_id`, `retweeted_status_user_id`,`retweeted_status_timestamp` -> These fields can be aggregated as pointers to WRD's original tweets. They cause an juxtaposition of duplicated variables in the dataset and should be dropped.
>
>* `floofer` -> Based on the definition of the various [dog stages]() a floofer is basically any dog. We're _already dealing_ with a dataset about dogs so there really isn't any insight we could pull from this description as a column on it's own.
>
> Pandas' `drop()`method will be effective for this operation.

##### Code

In [1145]:
#Set inplace=True to save your changes
df_copy_arch.drop(columns=['in_reply_to_status_id', 'in_reply_to_user_id','retweeted_status_id', 'retweeted_status_user_id','retweeted_status_timestamp', 'expanded_urls','floofer'], inplace=True)

##### Test

In [1147]:
df_copy_arch.head()

,timestamp,source,text,rating_numerator,rating_denominator,name,doggo,pupper,puppo
tweet_id,,,,,,,,,
892420643555336193,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,13,10,Phineas,None,None,None
892177421306343426,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,13,10,Tilly,None,None,None
891815181378084864,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,12,10,Archie,None,None,None
891689557279858688,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,13,10,Darla,None,None,None
891327558926688256,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,12,10,Franklin,None,None,None


>* We have obtained a better looking dataset without the unnecessary fields 🙂

##### Define

>* **(Quality issue)**: The four columns describing the "stage" the dog is in should be transposed into one column. 

##### Code

In [1148]:
df_copy_arch = df_copy_arch[['timestamp', 'source', 'text', 'rating_numerator', 'rating_denominator','name','pupper', 'puppo', 'doggo']]

In [1149]:
df_copy_arch.head()

,timestamp,source,text,rating_numerator,rating_denominator,name,pupper,puppo,doggo
tweet_id,,,,,,,,,
892420643555336193,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,13,10,Phineas,None,None,None
892177421306343426,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,13,10,Tilly,None,None,None
891815181378084864,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,12,10,Archie,None,None,None
891689557279858688,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,13,10,Darla,None,None,None
891327558926688256,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,12,10,Franklin,None,None,None


In [1150]:
df_copy_arch['dog_stage'] = pd.DataFrame(df_copy_arch.loc[:,['pupper', 'puppo', 'doggo']].apply(lambda x: '-'.join(x.values.astype(str)).lower(), axis=1))

In [1151]:
df_copy_arch.drop(columns=['pupper', 'puppo', 'doggo'], inplace=True)

In [1152]:
df_copy_arch.dog_stage.value_counts()

none-none-none       1985
pupper-none-none      245
none-none-doggo        84
none-puppo-none        29
pupper-none-doggo      12
none-puppo-doggo        1
Name: dog_stage, dtype: int64

In [1153]:
def getdogStage(x):
    
    x.dog_stage.mask(x.dog_stage == "none-none-none", "none", inplace=True)
    x.dog_stage.mask(x.dog_stage == "pupper-none-none", "pupper", inplace=True)
    x.dog_stage.mask(x.dog_stage == "none-none-doggo", "doggo", inplace=True)
    x.dog_stage.mask(x.dog_stage == "none-puppo-none", "puppo", inplace=True)
    x.dog_stage.mask(x.dog_stage == "pupper-none-doggo", "pupper-doggo", inplace=True)
    x.dog_stage.mask(x.dog_stage == "none-puppo-doggo", "puppo-doggo", inplace=True)
    
    return x

In [1154]:
getdogStage(df_copy_arch)

,timestamp,source,text,rating_numerator,rating_denominator,name,dog_stage
tweet_id,,,,,,,
892420643555336193,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,13,10,Phineas,none
892177421306343426,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,13,10,Tilly,none
891815181378084864,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,12,10,Archie,none
891689557279858688,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,13,10,Darla,none
891327558926688256,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,12,10,Franklin,none
...,...,...,...,...,...,...,...
666049248165822465,2015-11-16 00:24:50 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here we have a 1949 1st generation vulpix. Enj...,5,10,None,none
666044226329800704,2015-11-16 00:04:52 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is a purebred Piers Morgan. Loves to Netf...,6,10,a,none
666033412701032449,2015-11-15 23:21:54 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here is a very happy pup. Big fan of well-main...,9,10,a,none


##### Test

In [1155]:
df_copy_arch.dog_stage.value_counts()

none            1985
pupper           245
doggo             84
puppo             29
pupper-doggo      12
puppo-doggo        1
Name: dog_stage, dtype: int64

##### Define

> **(Quality Issue)** Representation of the data in the `source` column
>
>
>

##### Code

In [1156]:
df_copy_arch.source.value_counts()

<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>     2221
<a href="http://vine.co" rel="nofollow">Vine - Make a Scene</a>                          91
<a href="http://twitter.com" rel="nofollow">Twitter Web Client</a>                       33
<a href="https://about.twitter.com/products/tweetdeck" rel="nofollow">TweetDeck</a>      11
Name: source, dtype: int64

In [1157]:
def getSource(x):
    
    x.source.mask(x.source == '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>', 'Twitter for iPhone', inplace=True)
    x.source.mask(x.source == '<a href="http://vine.co" rel="nofollow">Vine - Make a Scene</a>" rel="nofollow">Twitter for iPhone</a>', 'Vine', inplace=True)
    x.source.mask(x.source == '<a href="http://twitter.com" rel="nofollow">Twitter Web Client</a>', 'Twitter Web Client', inplace=True)
    x.source.mask(x.source == '<a href="https://about.twitter.com/products/tweetdeck" rel="nofollow">TweetDeck</a>', 'Tweetdeck', inplace=True)

    return x

##### Test

In [1158]:
getSource(df_copy_arch)

,timestamp,source,text,rating_numerator,rating_denominator,name,dog_stage
tweet_id,,,,,,,
892420643555336193,2017-08-01 16:23:56 +0000,Twitter for iPhone,This is Phineas. He's a mystical boy. Only eve...,13,10,Phineas,none
892177421306343426,2017-08-01 00:17:27 +0000,Twitter for iPhone,This is Tilly. She's just checking pup on you....,13,10,Tilly,none
891815181378084864,2017-07-31 00:18:03 +0000,Twitter for iPhone,This is Archie. He is a rare Norwegian Pouncin...,12,10,Archie,none
891689557279858688,2017-07-30 15:58:51 +0000,Twitter for iPhone,This is Darla. She commenced a snooze mid meal...,13,10,Darla,none
891327558926688256,2017-07-29 16:00:24 +0000,Twitter for iPhone,This is Franklin. He would like you to stop ca...,12,10,Franklin,none
...,...,...,...,...,...,...,...
666049248165822465,2015-11-16 00:24:50 +0000,Twitter for iPhone,Here we have a 1949 1st generation vulpix. Enj...,5,10,None,none
666044226329800704,2015-11-16 00:04:52 +0000,Twitter for iPhone,This is a purebred Piers Morgan. Loves to Netf...,6,10,a,none
666033412701032449,2015-11-15 23:21:54 +0000,Twitter for iPhone,Here is a very happy pup. Big fan of well-main...,9,10,a,none


##### Define

**(Quality issue)**: the `timestamp` column has an incorrect datatype

##### Code

In [1159]:
df_copy_arch.timestamp

tweet_id
892420643555336193    2017-08-01 16:23:56 +0000
892177421306343426    2017-08-01 00:17:27 +0000
891815181378084864    2017-07-31 00:18:03 +0000
891689557279858688    2017-07-30 15:58:51 +0000
891327558926688256    2017-07-29 16:00:24 +0000
                                ...            
666049248165822465    2015-11-16 00:24:50 +0000
666044226329800704    2015-11-16 00:04:52 +0000
666033412701032449    2015-11-15 23:21:54 +0000
666029285002620928    2015-11-15 23:05:30 +0000
666020888022790149    2015-11-15 22:32:08 +0000
Name: timestamp, Length: 2356, dtype: object

In [1160]:
df_copy_arch.timestamp.str[:-6]

tweet_id
892420643555336193    2017-08-01 16:23:56
892177421306343426    2017-08-01 00:17:27
891815181378084864    2017-07-31 00:18:03
891689557279858688    2017-07-30 15:58:51
891327558926688256    2017-07-29 16:00:24
                             ...         
666049248165822465    2015-11-16 00:24:50
666044226329800704    2015-11-16 00:04:52
666033412701032449    2015-11-15 23:21:54
666029285002620928    2015-11-15 23:05:30
666020888022790149    2015-11-15 22:32:08
Name: timestamp, Length: 2356, dtype: object

In [1161]:
df_copy_arch.timestamp = df_copy_arch.timestamp.str[:-6]

In [1162]:
df_copy_arch.timestamp = pd.to_datetime(df_copy_arch.timestamp, yearfirst=True, infer_datetime_format=True)

##### Test

In [1163]:
df_copy_arch.timestamp

tweet_id
892420643555336193   2017-08-01 16:23:56
892177421306343426   2017-08-01 00:17:27
891815181378084864   2017-07-31 00:18:03
891689557279858688   2017-07-30 15:58:51
891327558926688256   2017-07-29 16:00:24
                             ...        
666049248165822465   2015-11-16 00:24:50
666044226329800704   2015-11-16 00:04:52
666033412701032449   2015-11-15 23:21:54
666029285002620928   2015-11-15 23:05:30
666020888022790149   2015-11-15 22:32:08
Name: timestamp, Length: 2356, dtype: datetime64[ns]

##### Define
> **(Quality issue)** `rating_denominator` and `rating_numerator`
>
>
>

##### Code

In [1164]:
df_copy_arch.rating_numerator.value_counts()

12      558
11      464
10      461
13      351
9       158
8       102
7        55
14       54
5        37
6        32
3        19
4        17
2         9
1         9
75        2
15        2
420       2
0         2
80        1
144       1
17        1
26        1
20        1
121       1
143       1
44        1
60        1
45        1
50        1
99        1
204       1
1776      1
165       1
666       1
27        1
182       1
24        1
960       1
84        1
88        1
Name: rating_numerator, dtype: int64

In [1170]:
for i in range(0, len(df_copy_arch)+0):
    rating = int(re.findall(r'\d+', df_copy_arch.rating_numerator[i].astype(str)[0:2])[0])
    df_copy_arch.rating_numerator[i] = rating

C:\Users\tevinaduma\AppData\Local\Temp\ipykernel_12372\3176933805.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_copy_arch.rating_numerator[i] = rating


In [1171]:
df_copy_arch.head()

,tweet_id,timestamp,source,text,rating_numerator,rating_denominator,name,dog_stage
0,892420643555336193,2017-08-01 16:23:56,Twitter for iPhone,This is Phineas. He's a mystical boy. Only eve...,13,10,Phineas,none
1,892177421306343426,2017-08-01 00:17:27,Twitter for iPhone,This is Tilly. She's just checking pup on you....,13,10,Tilly,none
2,891815181378084864,2017-07-31 00:18:03,Twitter for iPhone,This is Archie. He is a rare Norwegian Pouncin...,12,10,Archie,none
3,891689557279858688,2017-07-30 15:58:51,Twitter for iPhone,This is Darla. She commenced a snooze mid meal...,13,10,Darla,none
4,891327558926688256,2017-07-29 16:00:24,Twitter for iPhone,This is Franklin. He would like you to stop ca...,12,10,Franklin,none


##### Test

In [1138]:
df_copy_arch.reset_index(inplace=True)

In [1139]:
df_copy_arch.rating_numerator[0]

13

In [1135]:
df_copy_arch.rating_numerator.value_counts()

12      558
11      464
10      461
13      351
9       158
8       102
7        55
14       54
5        37
6        32
3        19
4        17
2         9
1         9
75        2
15        2
420       2
0         2
80        1
144       1
17        1
26        1
20        1
121       1
143       1
44        1
60        1
45        1
50        1
99        1
204       1
1776      1
165       1
666       1
27        1
182       1
24        1
960       1
84        1
88        1
Name: rating_numerator, dtype: int64

In [1134]:
df_copy_arch.head()

,timestamp,source,text,rating_numerator,rating_denominator,name,dog_stage
tweet_id,,,,,,,
892420643555336193,2017-08-01 16:23:56,Twitter for iPhone,This is Phineas. He's a mystical boy. Only eve...,13,10,Phineas,none
892177421306343426,2017-08-01 00:17:27,Twitter for iPhone,This is Tilly. She's just checking pup on you....,13,10,Tilly,none
891815181378084864,2017-07-31 00:18:03,Twitter for iPhone,This is Archie. He is a rare Norwegian Pouncin...,12,10,Archie,none
891689557279858688,2017-07-30 15:58:51,Twitter for iPhone,This is Darla. She commenced a snooze mid meal...,13,10,Darla,none
891327558926688256,2017-07-29 16:00:24,Twitter for iPhone,This is Franklin. He would like you to stop ca...,12,10,Franklin,none


##### Test

In [1172]:
df_copy_arch.rating_numerator.value_counts()

12    559
11    464
10    461
13    351
9     158
8     102
14     56
7      55
5      37
6      32
3      19
4      17
1       9
2       9
20      2
17      2
42      2
0       2
15      2
75      2
84      1
99      1
26      1
44      1
60      1
45      1
80      1
66      1
50      1
16      1
18      1
27      1
96      1
24      1
88      1
Name: rating_numerator, dtype: int64

In [1173]:
df_copy_arch.head()

,tweet_id,timestamp,source,text,rating_numerator,rating_denominator,name,dog_stage
0,892420643555336193,2017-08-01 16:23:56,Twitter for iPhone,This is Phineas. He's a mystical boy. Only eve...,13,10,Phineas,none
1,892177421306343426,2017-08-01 00:17:27,Twitter for iPhone,This is Tilly. She's just checking pup on you....,13,10,Tilly,none
2,891815181378084864,2017-07-31 00:18:03,Twitter for iPhone,This is Archie. He is a rare Norwegian Pouncin...,12,10,Archie,none
3,891689557279858688,2017-07-30 15:58:51,Twitter for iPhone,This is Darla. She commenced a snooze mid meal...,13,10,Darla,none
4,891327558926688256,2017-07-29 16:00:24,Twitter for iPhone,This is Franklin. He would like you to stop ca...,12,10,Franklin,none


<hr>

In [1184]:
#for i in range(0, len(df_copy_arch)): 
# df_copy_arch.text[i] = df_copy_arch.text[i].rsplit(' ',1)[0]

C:\Users\tevinaduma\AppData\Local\Temp\ipykernel_12372\543458027.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_copy_arch.text[i] = df_copy_arch.text[i].rsplit(' ',1)[0]


<hr>